In [1]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [7]:
def get_all_elements_names_prices_infinite_scroll():

    page_url = "https://www.pepper.pl/nowe"

    driver = webdriver.Chrome('./chromedriver') 
    driver.get(page_url) 
    time.sleep(3)
    scroll_pause_time = 1
    screen_height = driver.execute_script("return window.screen.height;")
    i = 1

    while True:
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(scroll_pause_time)
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        if (screen_height) * i > scroll_height:
            break

    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    list_all_articles_id = soup.find_all(attrs={'class': "cept-tt thread-link linkPlain thread-title--list js-thread-title"})
    list_all_items_info = soup.find_all(attrs={'class': "cept-tt thread-link linkPlain thread-title--list js-thread-title"})
    list_all_items_price = soup.find_all(attrs={'class': "thread-price text--b cept-tp size--all-l size--fromW3-xl"})

    #print(list_all_items)
    for i in list_all_items_info:
        print(i.get_text())
    
    for j in list_all_items_price:
        print(j.get_text())

    #print(soup.main.prettify())


get_all_elements_names_prices_infinite_scroll()

/tmp/ipykernel_269/2543345692.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


Pralka ELECTROLUX EW6FN428BP 8kg 1200 obr
Piwo somersby 10+10 gratis przy zakupach za min. 49 zł z kartą @Polomarket lub aplikacją
WRC 10 PS5
Ekspres ciśnieniowy SAECO SM8780/00 Xelsis Deluxe
Karma Purizon Adult dla kota, kurczak i ryba – bez zbóż + 1.5kg gratis
Letnia Wyprzedaż na AliExpress ze zniżką 4/30$
Gra Dying Light 2 (+2xdlc) + Dying Light The Following Enhanced Edition Bundle Epic Turcja
Myjka wysokociśnieniowa parkside phd 170
Pad Microsoft Xbox Series z adapterem bezprzewodowym
Hulajnoga Elektryczna KUGOO Kirin S1 Pro - 350W 7,5Ah 8'' 30km/h 30KM Napęd na tylne koło - czarna
Decathlon -20% (cashback) oraz sklep 5 10 15 (cashback 36%)
Koszulki polo Pier One - 5 PACK
Seagate Storage Expansion Card 1TB do Xbox Series X i S STJR1000400
Wideorejestrator SMARTGPS DVR-1300L GPS Quad HD, 160 stopni
Drukarka 3D SOVOL SV06 z Polski $255.66
Portfel kryptowalut Ledger Nano S Plus lub Nano X 30% taniej z oficjalnej strony producenta
Baseus Szybka ładowarka USB C 65 W, PD GaN3 Fast Wall 

In [15]:
def get_all_article_id():

    page_url = "https://www.pepper.pl/nowe"

    driver = webdriver.Chrome('./chromedriver') 
    driver.get(page_url) 
    time.sleep(3)

    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    articles = soup.find_all('article')
    for i in articles:
        article_id = i["id"]
        print(article_id)


get_all_article_id()

/tmp/ipykernel_269/853920369.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


thread_684687
thread_684684
thread_684686
thread_684685
thread_684683
thread_684682
thread_684681
thread_684680
thread_684679
thread_684574
thread_684620
thread_684676
thread_684674
thread_684673
thread_684672
thread_684670
thread_684666
thread_684669
thread_684668
thread_684667
